# Vector Database Workshop with **Weaviate**

## Introduction 

Welcome to this workshop about Vector Databases and Weaviate!

This notebook is a hands-on tutorial to be used alongside an interactive Weaviate workshop. In this workshop and with this notebook you: learn what vector search is, perform your first semantic search with a prepared demo dataset and build your own vector search application.


## Table of Contents
1. What is a Vector Database?
    1. Perform your first semantic search
2. Build your first vector search application
    1. Create a Weaviate cluster and connect to it
    2. Get data and analyze it
    3. Make a data schema
    4. Vectorize & upload data
    5. Query data
3. Advanced search with Weaviate:
    1. Data classification
    2. Multi-modal search
    3. Custom ML models

## 1. What is a Vector Database?

## 2. Build your first vector search application

### 2.1 Create a Weaviate cluster and connect to it

In order to build your own vector search application with Weaviate, you need a place to start and run the Weaviate instance. You can run Weaviate on you local machine or your cloud provider, or you can run a Weaviate instance using the Weaviate Cluster Service (WCS). If you want to run Weaviate yourself using Docker Compose or Kubernetes, follow the [installation steps in Weaviate's documentation](https://weaviate.io/developers/weaviate/current/getting-started/installation.html) to get a `docker-compose.yml` file and how to start it up. In this tutorial, we are using the WCS to start a Weaviate instance and interact with it using the console.

1. Create a WCS account on https://console.semi.technology/.

There are two options of how you can create a cluster once you have an account. You can either create one directly in the Console, or you can use the Weaviate Python Client to create a cluster from your code. We are going for the latter option here. We need to install the Weaviate Python Client in the environment first, and then we can use the WCS method to log in. 

In [3]:
import sys
!{sys.executable} -m pip install weaviate-client==3.4.2

Now let's import the package and log in with your WCS account.

In [4]:
from getpass import getpass # hide password
import weaviate # to communicate to the Weaviate instance
from weaviate.wcs import WCS

In order to authenticate to WCS or Weaviate instance (if Weaviate instance has Authentication enable) we need to create an Authentication object. At the moment it supports two types of authentication credentials: 
* Password credentials: `weaviate.auth.AuthClientPassword(username='WCS_ACCOUNT_EMAIL', password='WCS_ACCOUNT_PASSWORD')`
* Token credentials `weaviate.auth.AuthClientCredentials(client_secret=YOUR_SECRET_TOKEN)`

For WCS we will use the Password credentials.  

In [11]:
my_credentials = weaviate.auth.AuthClientPassword(username=input("User name: "), password=getpass('Password: '))


The my_credentials object contains your credentials so be careful not make it public.

In [12]:
my_wcs = WCS(my_credentials)

Now that we connected to WCS, we can `create`, `delete_cluster`, `get_clusters`, `get_cluster_config` and check the status of a cluster with `is_ready` method. You can check additional documentation [here](https://weaviate-python-client.readthedocs.io/en/latest/weaviate.wcs.html).

*If you want to check the prototype and docstring of any methods in a notebook, run this command: `object.method?`. You can also use the `help()` function.*<br>
Ex: `WCS.is_ready?` or `my_wcs.is_ready?` or `help(WCS.is_ready)`.

When creating a Weaviate cluster, we also need to define which *vectorizer* (retriever) module we will use. A list of available modules can be found [here](https://weaviate.io/developers/weaviate/current/modules/index.html). For this tutorial we use the [`text2vec-contextionary` module](https://weaviate.io/developers/weaviate/current/retriever-vectorizer-modules/text2vec-contextionary.html) to vectorize data and queries. This module is CPU optimized and thus convenient to use in this lightweight notebook and the free WCS cluster. The Contextionary is Weaviate's own language vectorizer. It gives context to the language used in your dataset. The most recent `text2vec-contextionary` is trained using [fastText](https://fasttext.cc/) on Wiki and CommonCrawl data.

Alternatively, you can choose to use one of the `text2vec-transformers` modules to vectorize text. Keep in mind that the free 'sandbox' Weaviate cluster that you get from WCS runs on CPU only, so Transformer models can be slower to import/vectorize data. If you want to experiment with a `text2vec-transformers` module, you are advised to run a Weaviate locally or on your cloud and enable GPU. You can change the `module` config parameter to for example (available models can be found [here](https://weaviate.io/developers/weaviate/current/retriever-vectorizer-modules/text2vec-transformers.html)):

```python
modules = {
    "name": "text2vec-transformers", 
    "tag": "sentence-transformers-paraphrase-MiniLM-L6-v2"
    } 
```

In [19]:
with_auth = False # set to true if you don't want your instance to be publically available
cluster_name = None # you can change this to some name you like, if kept to None a random name will be generated
modules = {
    "name": "text2vec-contextionary", # for this tutorial we use the text2vec-contextionary module to vectorize data and queries. This module is CPU optimized and thus convenient to use in this lightweight notebook and the free WCS cluster
    "tag": "en0.16.0-v1.0.0"
    } 

weaviate_url = my_wcs.create(cluster_name, with_auth=with_auth, modules=modules, wait_for_completion=True) 

weaviate_url

All resources of the cluster are ready.:                                   
100% |██████████|[01:02<00:00,  1.60%/s]


'https://ukeiveyqbfistrpw.semi.network'

**Congratulations! You have your first Weaviate cluster running!**

You can check the cluster at the given URL. You can check the meta information at `<weaviate_url>/v1/meta`, it's data schema at `<weaviate_url>/v1/schema` and the data objects at `<weaviate_url>/v1/objects`. 

We need to connect to the cluster with the Python client in order to create a data schema, import data and query data:

In [5]:
weaviate_url = 'https://ukeiveyqbfistrpw.semi.network'
client = weaviate.Client(weaviate_url)
client.is_ready()

True

### 2.2 Get data and analyze it

We set up a Weaviate instance, connected to it and have it ready for requests. Now it is time to take a step back and get some data and analyze it. 

This step, as for all the machine learning models, is the most important one. Here we have to decide what is relevant, what is important and what data structures/types to use.

In this example we are going to use news articles, which are pieces of unstructured textual data. For this we are going to need the `newspaper3k` package:

In [6]:
!{sys.executable} -m pip install newspaper3k

In the next code block we defined a function that retrieves articles from a newspaper. It saves the articles in `objects`, which consists of the article's `title`, `authors`, `title`, `word_count` and a UUID3 `id` generated from the article's URL.

In [7]:
import nltk # it is a dependency of newspaper3k
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/laura/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [31]:
import newspaper
import uuid
import json
from tqdm import tqdm

def get_articles_from_newspaper(
        news_url: str, 
        max_articles: int=100
    ) -> None:
    """
    Download and save newspaper articles as weaviate schemas.
    Parameters
    ----------
    newspaper_url : str
        Newspaper title.
    """
    
    objects = []
    
    # Build the actual newspaper    
    news_builder = newspaper.build(news_url, memoize_articles=False)
    
    if max_articles > news_builder.size():
        max_articles = news_builder.size()
    pbar = tqdm(total=max_articles)
    pbar.set_description(f"{news_url}")
    i = 0
    while len(objects) < max_articles and i < news_builder.size():
        article = news_builder.articles[i]
        try:
            article.download()
            article.parse()
            article.nlp()

            if (article.title != '' and \
                article.title is not None and \
                article.summary != '' and \
                article.summary is not None and\
                article.authors):

                # create an UUID for the article using its URL
                article_id = uuid.uuid3(uuid.NAMESPACE_DNS, article.url)

                # create the object
                objects.append({
                    'id': str(article_id),
                    'title': article.title,
                    'summary': article.summary,
                    'authors': article.authors,
                    'word_count': len(article.summary.split())
                })
                
                pbar.update(1)

        except:
            # something went wrong with getting the article, ignore it
            pass
        i += 1
    pbar.close()
    return objects

Let's download articles from some newspapers. Feel free to add more newspapers! 

In [32]:
data = []
data += get_articles_from_newspaper('https://www.theguardian.com/international')
data += get_articles_from_newspaper('http://cnn.com')

http://cnn.com: 100%|██████████| 100/100 [02:11<00:00,  1.32s/it]


### 2.3 Make a Weaviate data schema

We have some data downloaded. Before we can upload it to Weaviate, we need to create a [data schema](https://weaviate.io/developers/weaviate/current/data-schema/index.html). Adding a Weaviate data schema is not a requirement; if you don’t add a data schema, an automatic schema will be generated from the data that you import. But [defining a data schema yourself](https://weaviate.io/developers/weaviate/current/data-schema/schema-configuration.html) allows you to name classes, properties and relations between data entities yourself, so you have full control over the naming and structure, and also about configuration of modules and data vectorizers. 

A data schema consist of `classes`. Each `class` has a name and has `properties`. A `property` defines a data value that you can add to a `class`. 

Let's define a minimal schema for our Newspapers example:

In [22]:
schema = {
    "classes": [
        {
            "class": "Article", # name of the class
            "description": "An Article class to store the article summary and its authors", # a description of what this class represents
            "properties": [ # class properties
                {
                    "name": "title",
                    "dataType": ["string"],
                    "description": "The title of the article", 
                },
                {
                    "name": "summary",
                    "dataType": ["text"],
                    "description": "The summary of the article",
                },
                {
                    "name": "wordCount",
                    "dataType": ["int"],
                    "description": "The number of words in the article's summary",
                },
                {
                    "name": "hasAuthors",
                    "dataType": ["Author"],
                    "description": "The authors this article has",
                }
        
            ]
        }, {
            "class": "Author",
            "description": "An Author class to store the author information",
            "properties": [
                {
                    "name": "name",
                    "dataType": ["string"],
                    "description": "The name of the author", 
                },
                {
                    "name": "wroteArticles",
                    "dataType": ["Article"],
                    "description": "The articles of the author", 
                }
            ]
        }

 ]
}

In the class schema above we create a class named `Article` with the description `An Article class to store the article summary and its authors`. The description is there to explain the user what this class is about.

Also we define 4 properties: 
* `title` - The title of the article, of type `string` (case sensitive),
* `summary` - The summary of the article, of data type `text` (case insensitive), 
* `wordCount` - The amount of words in the summary of the article, of data type `int`, 
* `hasAuthor` - The authors of the article, of data type `Author`. The `Author` is NOT a primitive data type; this is how we make cross-references between data items. This way you can link your data objects in-between them and create a relation graph. In this case, we need to define the class `Author` as well. The list of primitive data types can be found [here](https://www.semi.technology/developers/weaviate/current/data-schema/datatypes.html).

**NOTE 1:** The properties should always be in cameCase format and starts with a lowercased word.<br>
**NOTE 2:** The property data type is always a list because it can accept more than one data type. 

Now that we decided on the data structure, we can tell Weaviate this schema. This can be done by accessing the `schema` attribute of the client. 

The schema can be created by using the [`.create()` method](https://weaviate-python-client.readthedocs.io/en/latest/weaviate.schema.html#weaviate.schema.Schema.create), this option creates multiple classes at once (useful if you have the whole schema). Alternatively you can use the [`.create_class()` method](https://weaviate-python-client.readthedocs.io/en/latest/weaviate.schema.html#weaviate.schema.Schema.create_class), this option creates only one class per call.

Also we can check if a schema is present or if a particular class schema is present with the `.contains()` method.

More about schema methods, click [here](https://weaviate-python-client.readthedocs.io/en/latest/weaviate.schema.html#module-weaviate.schema) for the Python docs or [here](https://weaviate.io/developers/weaviate/current/restful-api-references/schema.html) for the general schema docs of Weaviate.

Because we defined each class separately we should use the `.create_class()` method:

In [23]:
client.schema.create(schema)

Lets get the schema from weaviate and look what was created.

In [24]:
# helper function
def prettify(json_dict): 
    print(json.dumps(json_dict, indent=2))

In [25]:
prettify(client.schema.get())

{
  "classes": [
    {
      "class": "Article",
      "description": "An Article class to store the article summary and its authors",
      "invertedIndexConfig": {
        "bm25": {
          "b": 0.75,
          "k1": 1.2
        },
        "cleanupIntervalSeconds": 60,
        "stopwords": {
          "additions": null,
          "preset": "en",
          "removals": null
        }
      },
      "moduleConfig": {
        "text2vec-contextionary": {
          "vectorizeClassName": true
        }
      },
      "properties": [
        {
          "dataType": [
            "string"
          ],
          "description": "The title of the article",
          "moduleConfig": {
            "text2vec-contextionary": {
              "skip": false,
              "vectorizePropertyName": false
            }
          },
          "name": "title",
          "tokenization": "word"
        },
        {
          "dataType": [
            "text"
          ],
          "description": "The summary

You can always delete your entire schema with `client.schema.delete_all()`.

### 2.4 Vectorize & upload data

In the previous steps we have preprocessed our data and have created a Weaviate data schema. Now we're ready to add `Articles` and `Authors`.

Importing data to weaviate can be done in 2 different ways.

1. Adding object by object iteratively. This ca be done using the `data_object` object attribute of the client. This option requires one REST request per object, thus is slower than importing data in batches.
2. Adding objects in batches, with the  `Batch` class. This is efficient if you have large amounts of data to upload. The new class also supports 3 different cases of loading data in batches: a) Manually - the user has the absolute control when and how to add and create batches; b) Auto-create batches when full; c) Auto-create batches using dynamic batching, i.e. the batch size is adjusted every time it is created to avoid any Timeout errors.

We will use the second method to upload our data. 

First, we define functions `add_article()`, `add_author()` and `add_references()`. In this function we make clear how the data that we have saved in our `data` list will be stored in Weaviate, using the schema we defined in the previous step. 

In [27]:
from weaviate.batch import Batch # for the typing purposes
from weaviate.util import generate_uuid5


def add_article(batch: Batch, article_data: dict) -> str:
    
    article_object = {
        'title': article_data['title'],
        'wordCount': article_data['word_count'],
        'summary': article_data['summary'].replace('\n', '') # remove newline character
    }
    article_id = article_data['id']
    
    # add article to the batch
    batch.add_data_object( 
        data_object=article_object,
        class_name='Article',
        uuid=article_id
    )
    
    return article_id

def add_author(batch: Batch, author_name: str) -> str:
    
    # generate an UUID for the Author
    author_id = generate_uuid5(author_name)
    
    # add author to the batch
    batch.add_data_object(
        data_object={'name': author_name},
        class_name='Author',
        uuid=author_id
    )
    
    return author_id

def add_references(batch: Batch, article_id: str, author_id: str)-> None:
    # add references to the batch
    ## Author -> Article
    batch.add_reference(
        from_object_uuid=author_id,
        from_object_class_name='Author',
        from_property_name='wroteArticles',
        to_object_uuid=article_id
    )
    
    ## Article -> Author 
    batch.add_reference(
        from_object_uuid=article_id,
        from_object_class_name='Article',
        from_property_name='hasAuthors',
        to_object_uuid=author_id
    )

Next, we use the `batch()` method to upload the data to Weaviate. We are using a dynamic auto-create batch method. The batch method automatically creates objects and references when a batch is full, which is determined dynamically (depending on the object's size). 

In [33]:
client.batch.configure(batch_size=50, dynamic=True, callback=None)

with client.batch as batch:

    for i in data:

        # add article to the batch
        article_id = add_article(batch, i)

        for author in i['authors']:

            # add author to the batch
            author_id = add_author(batch, author)

            # add cross references to the batch
            add_references(batch, article_id=article_id, author_id=author_id)

If the data upload is successful, you are now ready to query the data!

### 2.5 Query data

#### 2.5.1 Query data using the GUI

#### 2.5.2 Query data using python